In [18]:
import numpy as np
import galois

Fungsi-fungsi yang digunakan

In [20]:
# Fungsi merubah bentuk ke matrik state
def baris(isi):
    temp2 = []
    for i in range(0,8,2):
        hex_2 = isi[i:i+2]
        temp2.append(hex_2)
    hasil=temp2
    return hasil

# Fungsi merubah plaintex menjadi hexadecimal
def rubah(isi):
    temp=""
    for k in isi:
        hexa=hex(ord(k))[2:]
        temp+=hexa
    return temp

# Fungsi merubah bentuk ke matrik state
def state(isi):
    temp=[]
    if len(isi) < 32:
        kosong=32-len(isi)
        isi+="0"*kosong
    temp2 = []
    for i in range(0,32,2):
        hex_2 = isi[i:i+2]
        temp2.append(hex_2)
    hasil=np.array(temp2).reshape(4,4).T
    return hasil

# Fungsi merubah bentuk array ke kalimat
def matriks_ke_kalimat(matriks):
    kalimat = ""
    for baris in matriks:
        for elemen in baris:
            kalimat += str(elemen)
    return kalimat.strip()
    
# Fungsi merubah hexadecimal menjadi biner
def hex_to_bin(hex_string):
    scale = 16 ## equals to hexadecimal
    num_of_bits = 4
    return bin(int(hex_string, scale))[2:].zfill(len(hex_string) * num_of_bits)

# Fungsi merubah biner menjadi hexadecimal
def bin_to_hex(bin_string):
    return hex(int(bin_string, 2))[2:].zfill(len(bin_string) // 4)

# Fungsi hexadecimal ke bilangan
def hex_to_int(isi):
    return int(isi,16)

Transformasi yang dipakai

In [5]:
# Fungsi melakukan xor
def xor(isi,isi1):
    isi1=hex_to_bin(isi1)
    isi=hex_to_bin(isi)
    result=""
    for i in range(len(isi)):
        if isi[i]==isi1[i]:
            result+="0"
        else:
            result+="1"
    hasil=bin_to_hex(result)
    return hasil

# Fungsi SubByte()
def SubBytes(a):
    bit_p=[0,1,1,0,0,0,1,1]
    c=bit_p[::-1]
    a=hex_to_int(a)
    polinomial=GF(a)
    bit_pol = bin(int(polinomial))[2:].zfill(8)
    bit_poli=bit_pol[::-1]
    if a!=0:
        invers=polinomial**(-1)
        bit_inv=bin(int(invers))[2:].zfill(8)
        bit_invers=bit_inv[::-1]
    else:
        bit_invers="00000000"
    b=[]
    for i in range(8):
        bb=int(c[i])+int(bit_invers[i])+int(bit_invers[((i+4)%8)])
        +int(bit_invers[((i+5)%8)])+int(bit_invers[((i+6)%8)])
        +int(bit_invers[((i+7)%8)])
        kk=bb%2
        b.append(kk)
    balik=b[::-1]
    kkk=''.join(map(str, balik))
    hasil=bin_to_hex(kkk)
    return hasil
    
# Fungsi ShitRows
def shitrows(sb):
    SR=[]
    for i in range(4):
        for j in range(4):
            sr=sb[i][(j+i)%4]
            SR.append(sr)
    SR=np.array(SR).reshape(4,4)
    return SR

# Fungsi MixColumns()
def mixcolumns(sr):
    hasil=[]
    a=np.array([["02","03","01","01"],
              ["01","02","03","01"],
              ["01","01","02","03"],
              ["03","01","01","02"]])
    result_matrix=[]
    
    for i in range(4):
        for j in range(4):
            h=GF(0)
            for k in range(4):
                h=h+GF(hex_to_int(a[i][k]))* GF(hex_to_int(sr[k][j]))
            bit_pol = bin(int(h))[2:].zfill(8)
            aa=bin_to_hex(bit_pol)
            result_matrix.append(aa)
    hasil=np.array(result_matrix).reshape(4,4)
    return hasil

Fungsi di Pembangkit Key

In [38]:
# Fungsi pergeseran 1 kali
def rotword(w):
    H=[]
    for i in range(4):
        H.append(w[(i+1)%4])
    return H

# Fungsi subword()
def subword(w):
    B=[]
    for i in w:
        B.append(SubBytes(i))
    return B

def keyexpansion(key):
    Rcon=[]
    for i in range(10):
        R1=hex_to_int("2")
        polinomial=GF(R1)**i
        bit_pol = bin(int(polinomial))[2:].zfill(8)
        bit_poli=bit_pol[::-1]
        h=bin_to_hex(bit_pol)+"00"*3
        Rcon.append(h)
        # print(R)
    w=[]
    for i in range(4):
        x=[key[4*i*2:4*i*2+2],key[4*i*2+2:4*i*2+2+2],key[4*i*2+4:4*i*2+2+4],key[4*i*2+6:4*i*2+2+6]]
        w.append(x)
    for i in range(4,44):
        temp=w[i-1]
        if i%4 ==0:
            temp=xor(matriks_ke_kalimat(subword(rotword(temp))),Rcon[int(i/4)-1])
        else:
            temp=matriks_ke_kalimat(temp)
        hasil=xor(matriks_ke_kalimat(w[i-4]),temp)
        w.append(baris(hasil))
    return w

In [28]:
GF=galois.GF(2**8,irreducible_poly=galois.Poly(
    [1, 0, 0, 0, 1, 1, 0, 1, 1],field=galois.GF(2)))
GF.repr("poly")

In [34]:
plaintext="00112233445566778899aabbccddeeff"
key="000102030405060708090a0b0c0d0e0f"

In [40]:
st=state(plaintext)
ky=state(key)
kunci=keyexpansion(key)

Proses looping

In [42]:
for i in range(11):
    kun=matriks_ke_kalimat(kunci[i*4:i*4+4])
    print(f'\nkunci untuk round ke-{i}: ',kun)


kunci untuk round ke-0:  000102030405060708090a0b0c0d0e0f

kunci untuk round ke-1:  d6aa74fdd2af72fadaa678f1d6ab76fe

kunci untuk round ke-2:  b692cf0b643dbdf1be9bc5006830b3fe

kunci untuk round ke-3:  b6ff744ed2c2c9bf6c590cbf0469bf41

kunci untuk round ke-4:  47f7f7bc95353e03f96c32bcfd058dfd

kunci untuk round ke-5:  3caaa3e8a99f9deb50f3af57adf622aa

kunci untuk round ke-6:  5e390f7df7a69296a7553dc10aa31f6b

kunci untuk round ke-7:  14f9701ae35fe28c440adf4d4ea9c026

kunci untuk round ke-8:  47438735a41c65b9e016baf4aebf7ad2

kunci untuk round ke-9:  549932d1f08557681093ed9cbe2c974e

kunci untuk round ke-10:  13111d7fe3944a17f307a78b4d2b30c5


Transformasi AddRoundKey

In [640]:
addR=[]
for i in range(4):
    for j in range(4):
        # print(st[j][i])
        # print(ky[j][i])
        hasil2=xor(st[j][i],ky[j][i])
        addR.append(hasil2)
addR=np.array(addR).reshape(4,4).T
rounde=matriks_ke_kalimat(addR.T)
print(addR)
print('\nRound 1 Start: ',rounde)

[['00' '40' '80' 'c0']
 ['10' '50' '90' 'd0']
 ['20' '60' 'a0' 'e0']
 ['30' '70' 'b0' 'f0']]

Round 1 Start:  00102030405060708090a0b0c0d0e0f0


Transformasi Subbyte()

In [644]:
SB=[]
for i in range(4):
    for j in range(4):
        hasil=SubBytes(addR[j][i])
        SB.append(hasil)
SB=np.array(SB).reshape(4,4).T
rounde=matriks_ke_kalimat(SB.T)
print(SB)
print('Round 1 Subbyte: ',rounde)

[['63' '09' 'cd' 'ba']
 ['ca' '53' '60' '70']
 ['b7' 'd0' 'e0' 'e1']
 ['04' '51' 'e7' '8c']]
Round 1 Subbyte:  63cab7040953d051cd60e0e7ba70e18c


ShiftRows()

In [646]:
SR=shitrows(SB)
rounde=matriks_ke_kalimat(SR.T)
print(SR)
print('Round 1 ShiftRows: ',rounde)

[['63' '09' 'cd' 'ba']
 ['53' '60' '70' 'ca']
 ['e0' 'e1' 'b7' 'd0']
 ['8c' '04' '51' 'e7']]
Round 1 ShiftRows:  6353e08c0960e104cd70b751bacad0e7


MixColumns()

In [674]:
MC=mixcolumns(SR)
rounde=matriks_ke_kalimat(MC.T)
print(MC)
print('Round 1 ShiftRows: ',rounde)

[['5f' '57' 'f7' '1d']
 ['72' 'f5' 'be' 'b9']
 ['64' 'bc' '3b' 'f9']
 ['15' '92' '29' '1a']]
Round 1 ShiftRows:  5f72641557f5bc92f7be3b291db9f91a


AddRoundKey

In [678]:
st=MC
ky=state(matriks_ke_kalimat(w[4:8]))

array([['5f', '57', 'f7', '1d'],
       ['72', 'f5', 'be', 'b9'],
       ['64', 'bc', '3b', 'f9'],
       ['15', '92', '29', '1a']], dtype='<U2')

In [680]:
addR=[]
for i in range(4):
    for j in range(4):
        hasil2=xor(st[j][i],ky[j][i])
        addR.append(hasil2)
addR=np.array(addR).reshape(4,4).T
rounde=matriks_ke_kalimat(addR.T)
print(addR)
print('\nRound 2 Start: ',rounde)

[['89' '85' '2d' 'cb']
 ['d8' '5a' '18' '12']
 ['10' 'ce' '43' '8f']
 ['e8' '68' 'd8' 'e4']]

Round 2 Start:  89d810e8855ace682d1843d8cb128fe4
